In [3]:
from restaurentpy.data import ReviewData

ReviewData(path='/Volumes/Macintosh HD/AI World/Review Data/').read_review(pat='xlsx')

AttributeError: 'ReviewData' object has no attribute 'pat'

In [15]:
import pandas as pd
# import os
import langdetect # Package to identify languages
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

# Read Data

In [12]:
df_review = pd.read_excel(
    '/Volumes/Macintosh HD/AI World/restaurentr/inst/extdata/inputdata/The Cheesecake Factory - Festival City.xlsx'
)

# Select columns need for Analysis
df_review = df_review[["review_datetime_utc", "review_text", "review_rating"]]

# Get Month year from datetime column
df_review['calendar_date'] = pd.to_datetime(df_review['review_datetime_utc']).apply(lambda x: x.strftime('%B-%Y')) 

df_review = df_review[["calendar_date", "review_text", "review_rating"]]

df_review.head()

,calendar_date,review_text,review_rating
0,December-2023,Love the lighting here and the vibe is amazing...,5
1,November-2023,Today is my birthday and first time having din...,5
2,November-2023,"If you love Cheesecake, then Cheesecake factor...",5
3,October-2023,I always get the same dish; the baja chicken t...,5
4,October-2023,One of my favorite restaurants. Quality of foo...,5


# Language Detection & Translate

In [13]:
def get_language(text):
    try:
        return langdetect.detect(text)
    except:
        return '<-- ERROR -->'

In [14]:
df_review['lang'] = df_review.review_text.map(get_language)

df_review.head()

,calendar_date,review_text,review_rating,lang
0,December-2023,Love the lighting here and the vibe is amazing...,5,en
1,November-2023,Today is my birthday and first time having din...,5,en
2,November-2023,"If you love Cheesecake, then Cheesecake factor...",5,en
3,October-2023,I always get the same dish; the baja chicken t...,5,en
4,October-2023,One of my favorite restaurants. Quality of foo...,5,en


In [21]:
df_review['lang'].value_counts()

lang
en               1368
ar                172
ro                 22
de                 20
fr                 16
af                 16
nl                 12
sw                 10
tr                 10
<-- ERROR -->      10
so                 10
id                  9
ca                  9
ru                  7
fa                  7
es                  7
ur                  6
it                  4
pl                  4
he                  4
et                  3
da                  3
fi                  3
sk                  3
ja                  3
sl                  2
ko                  2
tl                  2
sv                  2
cy                  2
hr                  2
no                  1
sq                  1
pt                  1
hu                  1
Name: count, dtype: int64